<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/5%20-%20Analisi%20del%20testo/named_entity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition
La Named Entity Recognition (NER) è l'operazione di livello superiore al POS Tagging e consiste nell'identificare la classe di appartenenza di una determinata parola. Esempi di classi possono essere: persone, organizzazioni, luoghi e quantità.
<br><br>
Entità è un'altra delle informazioni che Spacy ci restituisce, vediamo come ottenerla.
Importiamo Spacy e carichiamo il modulo per la lingua inglese.

In [0]:
import spacy

nlp = spacy.load("en_core_web_sm")

Creiamo un documento con una che contiene diverse entità, come una persona, un'organizzazione, una cifra e una data.

In [0]:
doc = nlp("Mark Zuckerberg acquired Whatsapp for 15 billions USD on 15 August 2014.")

Adesso possiamo accedere alle entità usando l'attributò *.ents*

In [14]:
doc.ents

(Mark Zuckerberg, Whatsapp, 15 billions USD, 15 August 2014)

Se l'attributo è vuoto, vuol dire che nel nostro testo non è presente (o non è stata rilevata) nessuna entità, possiamo stampare il tipo di entità usando l'attributo *.label_* di ogni entità.

In [15]:
print("TOKEN\t\tENTITA'")

for ent in doc.ents:
  print(ent.text+"\t\t"+ent.label_)

TOKEN		ENTITA'
Mark Zuckerberg		PERSON
Whatsapp		ORG
15 billions USD		MONEY
15 August 2014		DATE


Anche in questo caso possiamo usare la funzione *explain* per ottenere una descrizione dell'entità.

In [16]:
print("TOKEN\t\tENTITA'\t\tDESCRIZIONE")

for ent in doc.ents:
  print(ent.text+"\t\t"+ent.label_+"\t\t"+spacy.explain(ent.))

TOKEN		ENTITA'		DESCRIZIONE
Mark Zuckerberg		PERSON		People, including fictional
Whatsapp		ORG		Companies, agencies, institutions, etc.
15 billions USD		MONEY		Monetary values, including unit
15 August 2014		DATE		Absolute or relative dates or periods


Come vedi Spacy ha fatto un'ottimo lavoro, vediamo se fa lo stesso con un'esempio in Italiano.

In [4]:
!python -m spacy download it_core_news_sm


    Linking successful
    /usr/local/lib/python3.6/dist-packages/it_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/it_core_news_sm

    You can now load the model via spacy.load('it_core_news_sm')



In [34]:
nlp = spacy.load("it_core_news_sm")

doc = nlp("Il 15 gennaio 2020 Giuseppe Gullo ha venduto la sua azienda NIGMATICA a Mediaset per 10 milioni di euro ed è andato a vivere ad Helsinki con il suo gatto Elon.")

print("TOKEN\t\tENTITA'\t\tDESCRIZIONE")

for ent in doc.ents:
  print(ent.text+"\t\t"+ent.label_+"\t\t"+spacy.explain(ent.label_))

TOKEN		ENTITA'		DESCRIZIONE
Giuseppe Gullo		PER		Named person or family.
NIGMATICA		ORG		Companies, agencies, institutions, etc.
Mediaset		ORG		Companies, agencies, institutions, etc.
Helsinki		LOC		Non-GPE locations, mountain ranges, bodies of water
Elon		PER		Named person or family.


Come vedi anche in italiano il risultato è buono ma non eccellente come per l'inglese, infatti non è riuscito a riconoscere la data, se certe parole sono importanti per il nostro corpus di testo possiamo assegnargli una entità manualmente. Per farlo dobbiamo ottenere l'hash dell'entità corretta, in questo caso un'organizzazione.

In [37]:
org = doc.vocab.strings["DATE"]
org

388

E ci serve l'indice del token

In [38]:
index = doc.text.split().index("15")
index

1

Ora dobbiamo creare un'oggetto span che rappresenterà il nostro token, all'oggetto passeremo gli indici di inizio e fine del token all'interno della frase e il label corretto.

In [0]:
from spacy.tokens import Span

                        
nigma = Span(doc,index, index+3, label = org)

Adesso aggiungiamo lo span alle entità del nostro documento.

In [0]:
doc.ents = list(doc.ents) + [nigma]

Verifichiamo

In [43]:
print("TOKEN\t\tENTITA'\t\tDESCRIZIONE")

for ent in doc.ents:
  print(ent.text+"\t\t"+ent.label_+"\t\t"+spacy.explain(ent.label_))

TOKEN		ENTITA'		DESCRIZIONE
15 gennaio 2020		DATE		Absolute or relative dates or periods
Giuseppe Gullo		PER		Named person or family.
NIGMATICA		ORG		Companies, agencies, institutions, etc.
Mediaset		ORG		Companies, agencies, institutions, etc.
Helsinki		LOC		Non-GPE locations, mountain ranges, bodies of water
Elon		PER		Named person or family.


Adesso abbiamo anche la data !

## Visualizzazione delle entità
Spacy contiene un fantastico modulo per creare diversi tipi di visualizzazioni del testo chiamato Displacy. Importiamo displacy e utilizziamo per creare una rappresentazione visiva delle entità presenti nel testo.

In [33]:
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Jeff Bezos, founder and CEO of Amazon, an ecommerce company with headquarter in Seattle, became the world richest man on October 2017 with a net worth of 90 billions USD")

displacy.render(doc, style='ent', jupyter=True)

Se lavoriamo su Jupyter Notebook o Colaboratory utilizziamo il parametro jupyter=True per visualizzare il grafico direttamente all'interno del notebook, altrimenti l'ouput del rendering sarà del codice HTML che possiamo salvare e visualizzare dentro al browser.